In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [6]:
heart_data = pd.read_csv("heart.csv")
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [7]:
heart_data.shape

(303, 14)

In [8]:
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [9]:
heart_data['target'].value_counts()

target
1    165
0    138
Name: count, dtype: int64

In [10]:
X = heart_data.drop(columns='target', axis = 1)
Y = heart_data['target']
print(X)
print(Y)

     age  sex  cp  trestbps  chol  ...  exang  oldpeak  slope  ca  thal
0     63    1   3       145   233  ...      0      2.3      0   0     1
1     37    1   2       130   250  ...      0      3.5      0   0     2
2     41    0   1       130   204  ...      0      1.4      2   0     2
3     56    1   1       120   236  ...      0      0.8      2   0     2
4     57    0   0       120   354  ...      1      0.6      2   0     2
..   ...  ...  ..       ...   ...  ...    ...      ...    ...  ..   ...
298   57    0   0       140   241  ...      1      0.2      1   0     3
299   45    1   3       110   264  ...      0      1.2      1   0     3
300   68    1   0       144   193  ...      0      3.4      1   2     3
301   57    1   0       130   131  ...      1      1.2      1   1     3
302   57    0   1       130   236  ...      0      0.0      1   1     2

[303 rows x 13 columns]
0      1
1      1
2      1
3      1
4      1
      ..
298    0
299    0
300    0
301    0
302    0
Name: target

**Model selection**

In [35]:
models = [LogisticRegression(max_iter=10000), SVC(kernel= 'linear'), KNeighborsClassifier(), RandomForestClassifier(random_state=0)]

**CROSS VALIDATION**

In [18]:
def compare_models_cross_validation():
    for model in models:
        cv_score = cross_val_score(model, X, Y, cv = 5)
        mean_accuracy = np.mean(cv_score)*100
        mean_accuracy = round(mean_accuracy, 2)
        print('Cross validation accuracies for the ', model , ' = ', cv_score)
        print('Accuracy % of the ', model, ' = ' ,mean_accuracy)
        print('--------------------')

In [19]:
compare_models_cross_validation()

Cross validation accuracies for the  LogisticRegression(max_iter=10000)  =  [0.80327869 0.86885246 0.85245902 0.86666667 0.75      ]
Accuracy % of the  LogisticRegression(max_iter=10000)  =  82.83
--------------------
Cross validation accuracies for the  SVC(kernel='linear')  =  [0.81967213 0.8852459  0.80327869 0.86666667 0.76666667]
Accuracy % of the  SVC(kernel='linear')  =  82.83
--------------------
Cross validation accuracies for the  KNeighborsClassifier()  =  [0.60655738 0.6557377  0.57377049 0.73333333 0.65      ]
Accuracy % of the  KNeighborsClassifier()  =  64.39
--------------------
Cross validation accuracies for the  RandomForestClassifier(random_state=0)  =  [0.85245902 0.90163934 0.81967213 0.81666667 0.8       ]
Accuracy % of the  RandomForestClassifier(random_state=0)  =  83.81
--------------------


**Hyper parameter tuning**

In [38]:
model_list = [LogisticRegression(max_iter=10000), SVC(), KNeighborsClassifier(), RandomForestClassifier(random_state=0)]

In [20]:
model_hyperparameters = {
    'log_reg_hyperparameters':{
        'C': [1, 5, 10, 20]
    },
    'svc_hyperparameters': {
        'kernel': ['linear','poly', 'rbf', 'sigmoid'],
        'C': [1, 5, 10, 20]
    },
    'KNN_hyperparameters' : {
        'n_neighbors': [3, 5, 10]
    },
    'random_forest_hyperparameters' : {
        'n_estimators': [10, 20, 50, 100]
    }
}

In [21]:
type(model_hyperparameters)

dict

In [22]:
print(model_hyperparameters.keys())

dict_keys(['log_reg_hyperparameters', 'svc_hyperparameters', 'KNN_hyperparameters', 'random_forest_hyperparameters'])


In [23]:
print(model_hyperparameters.values())

dict_values([{'C': [1, 5, 10, 20]}, {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}, {'n_neighbors': [3, 5, 10]}, {'n_estimators': [10, 20, 50, 100]}])


In [24]:
model_keys = list(model_hyperparameters.keys())
print(model_keys)

['log_reg_hyperparameters', 'svc_hyperparameters', 'KNN_hyperparameters', 'random_forest_hyperparameters']


In [25]:
model_hyperparameters['svc_hyperparameters' ]

{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}

In [26]:
model_keys[2]

'KNN_hyperparameters'

In [27]:
model_hyperparameters[model_keys[0]]

{'C': [1, 5, 10, 20]}

In [36]:
def ModelSelection(list_of_models, hyperparameters_dictionary):
    result = []
    i = 0
    for model in list_of_models:
        key = model_keys[i]
        params = hyperparameters_dictionary[key]
        i+=1
        print(model)
        print(params)
        print('--------------------------------------')

        classifier = GridSearchCV(model, params, cv = 5)
        classifier.fit(X, Y)

        result.append(
            {
                'model used' : model, 
                'highest score': classifier.best_score_,
                'best hyperparameters': classifier.best_params_
            }
        )

    result_dataframe = pd.DataFrame(result, columns = ['model used', 'highest score', 'best hyperparameters'])

    return result_dataframe

In [39]:
ModelSelection(model_list, model_hyperparameters)

LogisticRegression(max_iter=10000)
{'C': [1, 5, 10, 20]}
--------------------------------------
SVC()
{'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1, 5, 10, 20]}
--------------------------------------
KNeighborsClassifier()
{'n_neighbors': [3, 5, 10]}
--------------------------------------
RandomForestClassifier(random_state=0)
{'n_estimators': [10, 20, 50, 100]}
--------------------------------------


,model used,highest score,best hyperparameters
0,LogisticRegression(max_iter=10000),0.831585,{'C': 5}
1,SVC(),0.828306,"{'C': 1, 'kernel': 'linear'}"
2,KNeighborsClassifier(),0.643880,{'n_neighbors': 5}
3,RandomForestClassifier(random_state=0),0.838087,{'n_estimators': 100}
